## Import some helpful libraries

In [1]:
print("    Version control\n------------------------")
import os, fnmatch, random, math, sys, datetime
from pathlib import Path
import numpy as np;              print("Numpy\t\t", np.__version__)
import matplotlib as mpl;        print("matplotlib\t", mpl.__version__)
import matplotlib.pyplot as plt
import nibabel as nib;           print("NiBabel\t\t {}".format(nib.__version__))
from nibabel.testing import data_path
import pandas as pd;             print("Pandas\t\t {}".format(pd.__version__))
import imageio;                  print("imageio\t\t {}".format(imageio.__version__))
import h5py;                     print("H5py\t\t {}".format(h5py.__version__))
import sklearn;                  print("Scikit-learn\t {}".format(sklearn.__version__))
import skimage;                  print("Scikit-image\t {}".format(skimage.__version__))
import tensorflow as tf;         print("TensorFlow\t {}".format(tf.__version__))
import keras as K;               print("Keras\t\t {}".format(K.__version__))
from tensorflow.keras import models, Input, Model
from tensorflow.keras.layers import Dense, Flatten, Reshape, BatchNormalization, Conv3D, MaxPooling3D, UpSampling3D
from tensorflow.keras.activations import relu, sigmoid
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.initializers import *
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
# %load_ext tensorboard       # %reload_ext tensorboard

    Version control
------------------------
Numpy		 1.19.4
matplotlib	 3.3.3
NiBabel		 3.2.0
Pandas		 1.1.4
imageio		 2.9.0
H5py		 2.10.0
Scikit-learn	 0.23.2
Scikit-image	 0.17.2
TensorFlow	 2.4.0
Keras		 2.4.3


## Loading Dataset

In [2]:
## Load train data
data_source = "Data/data_random_1/" 
sample_train_subset = np.loadtxt(os.path.join(data_source, "sample_train.csv"), dtype=str, delimiter=",")
train_data = np.load(os.path.join(data_source, "train.npy")).reshape(100,182,218,182,1)
print('train_data shape is {}'.format(train_data.shape))

## Load validation data
sample_val_subset = np.loadtxt(os.path.join(data_source, "sample_valid.csv"), dtype=str, delimiter=",")
valid_data = np.load(os.path.join(data_source, "valid.npy")).reshape(24,182,218,182,1)
print('valid_data shape is {}'.format(valid_data.shape))

train_data shape is (100, 182, 218, 182, 1)
valid_data shape is (24, 182, 218, 182, 1)


In [3]:
# print("There are", len(sample_train_subset), " subset of train samples are:\n")
# print(*sample_train_subset, sep='\t')
# print("\n--------------------------------------------------------------------------------\n")
# print("There are", len(sample_val_subset), " subset of Validation samples are:\n")
# print(*sample_val_subset, sep='\t')

In [4]:
# ## Showing one or all Training samples in three dimension (one middle slice per each dimension)

# def show_slices(slices):
#     fig, axes = plt.subplots(1, len(slices), figsize=(10,5))
#     for i, slice in enumerate(slices):
#         axes[i].imshow(slice.T, cmap="hot", origin="upper") # hot, Greys, gray
        
# # for m in range(train_data.shape[0]):
# for m in range(1):
#     slice_0 = train_data[m, 91, :, :, 0]
#     slice_1 = train_data[m, :, 109, :, 0]
#     slice_2 = train_data[m, :, :, 91, 0]
#     show_slices([slice_0, slice_1, slice_2])
#     plt.suptitle(sample_train_subset[m], x=0.5, y=0.9)

In [5]:
# ## Showing one or all Validation samples in three dimension (one middle slice per each dimension)

# # for m in range(valid_data.shape[0]):
# for m in range(1):
#     slice_0 = valid_data[m, 91, :, :, 0]
#     slice_1 = valid_data[m, :, 109, :, 0]
#     slice_2 = valid_data[m, :, :, 91, 0]
#     show_slices([slice_0, slice_1, slice_2])
#     plt.suptitle(sample_val_subset[m], x=0.5, y=1)

## Model Design

In [6]:
## Test simple model:

IMAGE_HEIGHT = train_data.shape[1]
IMAGE_WIDTH = train_data.shape[2]
IMAGE_DEPTH = train_data.shape[3]
BATCH_SIZE = 1
EPOCHS = 1
data_shape = [1, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, 1]
input_shape = [BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, 1]
## Encoder
input_img = Input(shape=(182, 218, 182, 1), name='Input')
x = Conv3D(filters=1, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv1')(input_img)
x = BatchNormalization(name='BN_Conv1')(x)
## Latent Features
shape_before_flattening = tf.keras.backend.int_shape(x)
x = Flatten(name='Flat')(x)
encoded = x
x = Reshape(shape_before_flattening[1:], name='UnFlat')(x)
## Decoder
x = Conv3D(filters=1, kernel_size=1, padding='valid', activation='relu', name='DeConv6')(x)
x = BatchNormalization(name='BN_DeConv1')(x)
x = UpSampling3D(size=(2, 2, 2), name='UpSampling1')(x)
decoded = Conv3D(filters=1, kernel_size=3, padding='same', activation='sigmoid', name='Output')(x)
model_CAE = Model(inputs=input_img, outputs=decoded, name='AutoEncoder')
model_CAE.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model_CAE.summary()

Model: "AutoEncoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 182, 218, 182, 1) 0         
_________________________________________________________________
Conv1 (Conv3D)               (None, 91, 109, 91, 1)    28        
_________________________________________________________________
BN_Conv1 (BatchNormalization (None, 91, 109, 91, 1)    4         
_________________________________________________________________
Flat (Flatten)               (None, 902629)            0         
_________________________________________________________________
UnFlat (Reshape)             (None, 91, 109, 91, 1)    0         
_________________________________________________________________
DeConv6 (Conv3D)             (None, 91, 109, 91, 1)    2         
_________________________________________________________________
BN_DeConv1 (BatchNormalizati (None, 91, 109, 91, 1)    

In [7]:
# ## Define parameters:

# IMAGE_HEIGHT = train_data.shape[1]
# IMAGE_WIDTH = train_data.shape[2]
# IMAGE_DEPTH = train_data.shape[3]
# BATCH_SIZE = 1
# EPOCHS = 1
# data_shape = [1, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, 1]
# input_shape = [BATCH_SIZE, IMAGE_HEIGHT, IMAGE_WIDTH, IMAGE_DEPTH, 1]
# print("input-layer shape:", input_shape)

# ## Encoder
# input_img = Input(shape=(182, 218, 182, 1), name='Input')
# x = Conv3D(filters=128, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv1')(input_img)
# x = BatchNormalization(name='BN_Conv1')(x)
# x = Conv3D(filters=64, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv2')(x)
# x = BatchNormalization(name='BN_Conv2')(x)
# x = Conv3D(filters=32, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv3')(x)
# x = BatchNormalization(name='BN_Conv3')(x)
# x = Conv3D(filters=16, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv4')(x)
# x = BatchNormalization(name='BN_Conv4')(x)
# x = Conv3D(filters=8, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv5')(x)
# x = BatchNormalization(name='BN_Conv5')(x)
# x = Conv3D(filters=4, kernel_size=3, strides=2, padding='same', activation='relu', name='Conv6')(x)
# x = BatchNormalization(name='BN_Conv6')(x)

# ## Latent Features
# shape_before_flattening = tf.keras.backend.int_shape(x)
# x = Flatten(name='LF')(x)
# # init = VarianceScaling(scale=1. / 3., mode='fan_in', distribution='uniform')
# # encoded = Dense(50, kernel_initializer=init, activation='relu', name='encoded')(x)
# # encoded = Dense(50, activation='relu', name='encoded')(x)
# encoded = x
# # x = BatchNormalization()(encoded)
# # x = Dense(np.prod(shape_before_flattening[1:]), activation='relu', kernel_initializer=init)(encoded)
# # x = Dense(np.prod(shape_before_flattening[1:]), activation='relu')(encoded)
# x = Reshape(shape_before_flattening[1:], name='UnFlat')(x)

# ## Decoder
# x = Conv3D(filters=4, kernel_size=3, padding='same', activation='relu', name='DeConv1')(x)
# x = BatchNormalization(name='BN_DeConv1')(x)
# x = UpSampling3D(size=(2, 2, 2), name='UpSampling1')(x)
# x = Conv3D(filters=8, kernel_size=(1,2,1), padding='valid', activation='relu', name='DeConv2')(x)
# x = BatchNormalization(name='BN_DeConv2')(x)
# x = UpSampling3D(size=(2, 2, 2), name='UpSampling2')(x)
# x = Conv3D(filters=16, kernel_size=3, padding='same', activation='relu', name='DeConv3')(x)
# x = BatchNormalization(name='BN_DeConv3')(x)
# x = UpSampling3D(size=(2, 2, 2), name='UpSampling3')(x)
# x = Conv3D(filters=32, kernel_size=(2,1,2), padding='valid', activation='relu', name='DeConv4')(x)
# x = BatchNormalization(name='BN_DeConv4')(x)
# x = UpSampling3D(size=(2, 2, 2), name='UpSampling4')(x)
# x = Conv3D(filters=64, kernel_size=(1,2,1), padding='valid', activation='relu', name='DeConv5')(x)
# x = BatchNormalization(name='BN_DeConv5')(x)
# x = UpSampling3D(size=(2, 2, 2), name='UpSampling5')(x)
# x = Conv3D(filters=128, kernel_size=2, padding='valid', activation='relu', name='DeConv6')(x)
# x = BatchNormalization(name='BN_DeConv6')(x)
# x = UpSampling3D(size=(2, 2, 2), name='UpSampling6')(x)
# decoded = Conv3D(filters=1, kernel_size=3, padding='same', activation='sigmoid', name='Output')(x)

# model_CAE = Model(inputs=input_img, outputs=decoded, name='AutoEncoder')
# ## optimizer=rmsprop, sgd
# model_CAE.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
# model_CAE.summary()

## Model Training

In [8]:
## Start time:
from datetime import datetime
start_time = datetime.now().strftime("%Y_%m_%d___%H_%M"); print("\nStart Time =", start_time, "\n")

## Model Fit
model_CAE.load_weights(os.path.join("Weights/L100___2021_01_01___23_46.hdf5"), by_name=True)
model_checkpoint_callback = ModelCheckpoint(filepath=os.path.join("Check/L100___" + start_time), save_weights_only=True, save_best_only=True, monitor='val_loss', mode='max', verbose=1) 
tb_callback = TensorBoard(os.path.join("Logs/L100___" + start_time), histogram_freq=1)
# early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=5, mode='auto')
model_CAE.fit(train_data[0:10,:], train_data[0:10,:], validation_data=(valid_data, valid_data), epochs=EPOCHS, batch_size=BATCH_SIZE, shuffle=True, callbacks=[tb_callback, model_checkpoint_callback], verbose=1)
model_CAE.save_weights(os.path.join("Weights/L100___" + start_time + ".hdf5"))

## End time:
# from datetime import datetime
end_time = datetime.now().strftime("%Y_%m_%d____%H_%M"); print("\nEnd Time =", end_time)


Start Time = 2021_01_02___16_29 

10/10 [==============================] - 78s 8s/step - loss: 0.0268 - accuracy: 0.6888 - val_loss: 0.0155 - val_accuracy: 0.7067

Epoch 00001: val_loss improved from -inf to 0.01552, saving model to Check/L100___2021_01_02___16_29

End Time = 2021_01_02____16_30


In [9]:
test_data = valid_data[0,:].reshape(1, 182, 218, 182, 1)
reconstructed = model_CAE.predict(test_data)

In [10]:
# for m in range(1):
#     slice_0 = reconstructed[m, 91, :, :, 0]
#     slice_1 = reconstructed[m, :, 109, :, 0]
#     slice_2 = reconstructed[m, :, :, 91, 0]
#     show_slices([slice_0, slice_1, slice_2])

In [11]:
# print('\ntrain_data[0,100,100:105,100]\n\n {}'.format(train_data[0,100,100:105,100]),'\n')
# print('\nReconstructed_data[0,100,100:105,100]\n\n {}'.format(reconstructed[0,100,100:105,100]),'\n')

In [12]:
# h5_file = h5py.File(os.path.join("Weights/L100___" + start_time + ".hdf5"), 'r')
# # h5_file = h5py.File(os.path.join("Weights/w_Pegasus_128.hdf5"), 'r')
# Layer_size = len(list(h5_file.keys()))
# Layer_names = list(h5_file.keys())
# print("There are", Layer_size, "layers in this model as:\n\n", Layer_names,'\n')

# for l in range(5):  #Layer_size
#     print('==========================================================\n')
#     layers = h5_file[Layer_names[l]]
# #     print("Layer", l+1, "-----", layers)
#     W = layers[Layer_names[l]]['kernel:0']
#     print('Layer', l+1, ':', list(h5_file.keys())[l], '\tWeights\' shape: {}'.format(W.shape), '\n')
# #     print('\nWeights[1][1][1]: {}'.format(W[1][1][1]))
    
#     Kernel_1 = W.shape[0]
#     Kernel_2 = W.shape[1]
#     Kernel_3 = W.shape[2]
#     Kernel_all = np.zeros([Kernel_1, Kernel_2, Kernel_3])
#     for f in range(2):   # W.shape[4]
#         for x in range(Kernel_1):
#             for y in range(Kernel_2):
#                 for z in range(Kernel_3):
#                     Kernel_all[x][y][z] = (W[x][y][z])[0][0]
#         print('\nWeights of kernel', f+1, 'of', W.shape[4], ':\n\n', Kernel_all)

In [13]:
# tensorboard --logdir=Logs       # http://localhost:6006/

In [14]:
from keras.engine.topology import Layer, InputSpec
class ClusteringLayer(Layer):
    """
    Clustering layer converts input sample (feature) to soft label, i.e. a vector that represents the probability of the
    sample belonging to each cluster. The probability is calculated with student's t-distribution.

    # Example
    ```
        model.add(ClusteringLayer(n_clusters=10))
    ```
    # Arguments
        n_clusters: number of clusters.
        weights: list of Numpy array with shape `(n_clusters, n_features)` witch represents the initial cluster centers.
        alpha: parameter in Student's t-distribution. Default to 1.0.
    # Input shape
        2D tensor with shape: `(n_samples, n_features)`.
    # Output shape
        2D tensor with shape: `(n_samples, n_clusters)`.
    """

    def __init__(self, n_clusters, weights=None, alpha=1.0, **kwargs):
        if 'input_shape' not in kwargs and 'input_dim' in kwargs:
            kwargs['input_shape'] = (kwargs.pop('input_dim'),)
        super(ClusteringLayer, self).__init__(**kwargs)
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.initial_weights = weights
        self.input_spec = InputSpec(ndim=2)

    def build(self, input_shape):
        assert len(input_shape) == 2
        input_dim = input_shape[1]
        self.input_spec = InputSpec(dtype=K.floatx(), shape=(None, input_dim))
        self.clusters = self.add_weight((self.n_clusters, input_dim), initializer='glorot_uniform', name='clusters')
        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights
        self.built = True

    def call(self, inputs, **kwargs):
        """ student t-distribution, as same as used in t-SNE algorithm.
                 q_ij = 1/(1+dist(x_i, u_j)^2), then normalize it.
        Arguments:
            inputs: the variable containing data, shape=(n_samples, n_features)
        Return:
            q: student's t-distribution, or soft labels for each sample. shape=(n_samples, n_clusters)
        """
        q = 1.0 / (1.0 + (K.sum(K.square(K.expand_dims(inputs, axis=1) - self.clusters), axis=2) / self.alpha))
        q **= (self.alpha + 1.0) / 2.0
        q = K.transpose(K.transpose(q) / K.sum(q, axis=1))
        return q

    def compute_output_shape(self, input_shape):
        assert input_shape and len(input_shape) == 2
        return input_shape[0], self.n_clusters

    def get_config(self):
        config = {'n_clusters': self.n_clusters}
        base_config = super(ClusteringLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
class DCEC(object):
    def __init__(self,
                 input_shape,
                 filters=[32, 64, 128, 10],
                 n_clusters=10,
                 alpha=1.0):

        super(DCEC, self).__init__()

        self.n_clusters = n_clusters
        self.input_shape = input_shape
        self.alpha = alpha
        self.pretrained = False
        self.y_pred = []

        self.cae = CAE(input_shape, filters)
        hidden = self.cae.get_layer(name='embedding').output
        self.encoder = Model(inputs=self.cae.input, outputs=hidden)

        # Define DCEC model
        clustering_layer = ClusteringLayer(self.n_clusters, name='clustering')(hidden)
        self.model = Model(inputs=self.cae.input,
                           outputs=[clustering_layer, self.cae.output])

    def pretrain(self, x, batch_size=256, epochs=200, optimizer='adam', save_dir='results/temp'):
        print('...Pretraining...')
        self.cae.compile(optimizer=optimizer, loss='mse')
        from keras.callbacks import CSVLogger
        csv_logger = CSVLogger(args.save_dir + '/pretrain_log.csv')

        # begin training
        t0 = time()
        self.cae.fit(x, x, batch_size=batch_size, epochs=epochs, callbacks=[csv_logger])
        print('Pretraining time: ', time() - t0)
        self.cae.save(save_dir + '/pretrain_cae_model.h5')
        print('Pretrained weights are saved to %s/pretrain_cae_model.h5' % save_dir)
        self.pretrained = True

    def load_weights(self, weights_path):
        self.model.load_weights(weights_path)

    def extract_feature(self, x):  # extract features from before clustering layer
        return self.encoder.predict(x)

    def predict(self, x):
        q, _ = self.model.predict(x, verbose=0)
        return q.argmax(1)

    @staticmethod
    def target_distribution(q):
        weight = q ** 2 / q.sum(0)
        return (weight.T / weight.sum(1)).T

    def compile(self, loss=['kld', 'mse'], loss_weights=[1, 1], optimizer='adam'):
        self.model.compile(loss=loss, loss_weights=loss_weights, optimizer=optimizer)

    def fit(self, x, y=None, batch_size=256, maxiter=2e4, tol=1e-3,
            update_interval=140, cae_weights=None, save_dir='./results/temp'):

        print('Update interval', update_interval)
        save_interval = x.shape[0] / batch_size * 5
        print('Save interval', save_interval)

        # Step 1: pretrain if necessary
        t0 = time()
        if not self.pretrained and cae_weights is None:
            print('...pretraining CAE using default hyper-parameters:')
            print('   optimizer=\'adam\';   epochs=200')
            self.pretrain(x, batch_size, save_dir=save_dir)
            self.pretrained = True
        elif cae_weights is not None:
            self.cae.load_weights(cae_weights)
            print('cae_weights is loaded successfully.')

        # Step 2: initialize cluster centers using k-means
        t1 = time()
        print('Initializing cluster centers with k-means.')
        kmeans = KMeans(n_clusters=self.n_clusters, n_init=20)
        self.y_pred = kmeans.fit_predict(self.encoder.predict(x))
        y_pred_last = np.copy(self.y_pred)
        self.model.get_layer(name='clustering').set_weights([kmeans.cluster_centers_])

        # Step 3: deep clustering
        # logging file
        import csv, os
        if not os.path.exists(save_dir):
            os.makedirs(save_dir)
        logfile = open(save_dir + '/dcec_log.csv', 'w')
        logwriter = csv.DictWriter(logfile, fieldnames=['iter', 'acc', 'nmi', 'ari', 'L', 'Lc', 'Lr'])
        logwriter.writeheader()

        t2 = time()
        loss = [0, 0, 0]
        index = 0
        for ite in range(int(maxiter)):
            if ite % update_interval == 0:
                q, _ = self.model.predict(x, verbose=0)
                p = self.target_distribution(q)  # update the auxiliary target distribution p

                # evaluate the clustering performance
                self.y_pred = q.argmax(1)
                if y is not None:
                    acc = np.round(metrics.acc(y, self.y_pred), 5)
                    nmi = np.round(metrics.nmi(y, self.y_pred), 5)
                    ari = np.round(metrics.ari(y, self.y_pred), 5)
                    loss = np.round(loss, 5)
                    logdict = dict(iter=ite, acc=acc, nmi=nmi, ari=ari, L=loss[0], Lc=loss[1], Lr=loss[2])
                    logwriter.writerow(logdict)
                    print('Iter', ite, ': Acc', acc, ', nmi', nmi, ', ari', ari, '; loss=', loss)

                # check stop criterion
                delta_label = np.sum(self.y_pred != y_pred_last).astype(np.float32) / self.y_pred.shape[0]
                y_pred_last = np.copy(self.y_pred)
                if ite > 0 and delta_label < tol:
                    print('delta_label ', delta_label, '< tol ', tol)
                    print('Reached tolerance threshold. Stopping training.')
                    logfile.close()
                    break

            # train on batch
            if (index + 1) * batch_size > x.shape[0]:
                loss = self.model.train_on_batch(x=x[index * batch_size::],
                                                 y=[p[index * batch_size::], x[index * batch_size::]])
                index = 0
            else:
                loss = self.model.train_on_batch(x=x[index * batch_size:(index + 1) * batch_size],
                                                 y=[p[index * batch_size:(index + 1) * batch_size],
                                                    x[index * batch_size:(index + 1) * batch_size]])
                index += 1

            # save intermediate model
            if ite % save_interval == 0:
                # save DCEC model checkpoints
                print('saving model to:', save_dir + '/dcec_model_' + str(ite) + '.h5')
                self.model.save_weights(save_dir + '/dcec_model_' + str(ite) + '.h5')

            ite += 1

        # save the trained model
        logfile.close()
        print('saving model to:', save_dir + '/dcec_model_final.h5')
        self.model.save_weights(save_dir + '/dcec_model_final.h5')
        t3 = time()
        print('Pretrain time:  ', t1 - t0)
        print('Clustering time:', t3 - t1)
        print('Total time:     ', t3 - t0)

In [15]:
model_CA = Model(inputs=input_img, outputs=encoded, name='Encoder')
model_CA.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
model_CA.summary()

Model: "Encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 182, 218, 182, 1) 0         
_________________________________________________________________
Conv1 (Conv3D)               (None, 91, 109, 91, 1)    28        
_________________________________________________________________
BN_Conv1 (BatchNormalization (None, 91, 109, 91, 1)    4         
_________________________________________________________________
Flat (Flatten)               (None, 902629)            0         
Total params: 32
Trainable params: 30
Non-trainable params: 2
_________________________________________________________________


In [ ]:
n_clusters = 2
clustering_layer = ClusteringLayer(n_clusters, name='clustering')(model_CA.output)

In [ ]:
C_model = Model(inputs=encoder.input, outputs=clustering_layer)

In [17]:
# !conda update -c conda-forge tensorflow

In [ ]:
!conda update -c conda-forge keras

Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: failed
Solving environment: \ 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
Examining conflict for anaconda glueviz: : 55it [6:12:56, 421.91s/it]                                                                                                                                                  | 